## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
import keras_tuner as kt
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
dropped_df = application_df.drop(['EIN', 'NAME'], axis = 1)
dropped_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [5]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
# Get list of categorical variables
app_cat = dropped_df.dtypes[dropped_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
dropped_df[app_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [9]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

other_values = dropped_df['APPLICATION_TYPE'].value_counts().loc[lambda x: x<250]
application_types_to_replace = other_values.index.tolist()
# Replace in dataframe
for app in application_types_to_replace:
    dropped_df['APPLICATION_TYPE'] = dropped_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
dropped_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Look at CLASSIFICATION value counts for binning
dropped_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1570        1
C2380        1
C1245        1
C6100        1
C1248        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [6]:
# You may find it helpful to look at CLASSIFICATION value counts >1
dropped_df['CLASSIFICATION'].value_counts().loc[lambda x: x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C4100        6
C2400        6
C1720        6
C1257        5
C1600        5
C0           3
C1260        3
C2710        3
C1234        2
C1246        2
C1267        2
C1256        2
C3200        2
Name: CLASSIFICATION, dtype: int64

In [12]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

other_cls = dropped_df['CLASSIFICATION'].value_counts().loc[lambda x: x<3000]
classifications_to_replace = other_cls.index.tolist()
# Replace in dataframe
for cls in classifications_to_replace:
    dropped_df['CLASSIFICATION'] = dropped_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
dropped_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
df_final = pd.get_dummies(dropped_df, prefix='', prefix_sep='',columns=['APPLICATION_TYPE',
                                                                        'AFFILIATION',
                                                                        'CLASSIFICATION',
                                                                        'USE_CASE',
                                                                        'ORGANIZATION',
                                                                        'INCOME_AMT',
                                                                        'SPECIAL_CONSIDERATIONS'])
df_final.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,Other,T10,T19,T3,T4,T5,T6,...,1-9999,10000-24999,100000-499999,10M-50M,1M-5M,25000-99999,50M+,5M-10M,N,Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [14]:
# Split our preprocessed data into our features and target arrays
y = df_final.IS_SUCCESSFUL.values
X = df_final.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

# Model 1

In [45]:
# increase number of layers, neurons and change activation from relu to sigmoin in 2nd and 3rd layers
# Initialise the Sequential class
nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=44, input_dim=len(X_train[0]), activation="relu"))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=44, activation="sigmoid"))

# Third hidden layer
nn1.add(tf.keras.layers.Dense(units=44, activation="sigmoid"))
# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 44)                1848      
                                                                 
 dense_22 (Dense)            (None, 44)                1980      
                                                                 
 dense_23 (Dense)            (None, 44)                1980      
                                                                 
 dense_24 (Dense)            (None, 1)                 45        
                                                                 
Total params: 5,853
Trainable params: 5,853
Non-trainable params: 0
_________________________________________________________________


In [46]:
# Compile the model 1
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [47]:
# Train the model
import keras
from keras.callbacks import Callback
mc = keras.callbacks.ModelCheckpoint('opt_weights{epoch:08d}.h5',save_weights_only=True, period=5)
fit_model = nn1.fit(X_train_scaled,y_train,callbacks=[mc],epochs=20)

Epoch 1/20
804/804 [==============================] - 1s 972us/step - loss: 0.5907 - accuracy: 0.7038
Epoch 2/20
804/804 [==============================] - ETA: 0s - loss: 0.5627 - accuracy: 0.7276 ETA: 0s - loss: 0.5605 - accu - 1s 979us/step - loss: 0.5627 - accuracy: 0.7285
Epoch 3/20
804/804 [==============================] - 1s 1ms/step - loss: 0.5602 - accuracy: 0.7283: 0s - loss: 0.5606 - ac
Epoch 4/20
804/804 [==============================] - 1s 959us/step - loss: 0.5582 - accuracy: 0.73000s - loss: 0.5627 - 
Epoch 5/20
804/804 [==============================] - 1s 966us/step - loss: 0.5573 - accuracy: 0.7287
Epoch 6/20
804/804 [==============================] - 1s 1ms/step - loss: 0.5561 - accuracy: 0.7297
Epoch 7/20
804/804 [==============================] - 1s 832us/step - loss: 0.5556 - accuracy: 0.7304
Epoch 8/20
804/804 [==============================] - 1s 794us/step - loss: 0.5543 - accuracy: 0.7306
Epoch 9/20
804/804 [==============================] - 1s 924us/step - 

In [48]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5565 - accuracy: 0.7238 - 256ms/epoch - 954us/step
Loss: 0.5564873814582825, Accuracy: 0.7238484025001526


# Model 2

In [37]:
# substantially increase number of neurons in hidden layers and change the activation back to relu
# Initialise the Sequential class
nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=70, input_dim=len(X_train[0]), activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=70, activation="relu"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=70, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 70)                2940      
                                                                 
 dense_16 (Dense)            (None, 70)                4970      
                                                                 
 dense_17 (Dense)            (None, 70)                4970      
                                                                 
 dense_18 (Dense)            (None, 1)                 71        
                                                                 
Total params: 12,951
Trainable params: 12,951
Non-trainable params: 0
_________________________________________________________________


In [38]:
# Compile the model 2
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [39]:
# Train the model
import keras
from keras.callbacks import Callback
mc = keras.callbacks.ModelCheckpoint('opt_weights{epoch:08d}.h5',save_weights_only=True, period=5)
fit_model = nn2.fit(X_train_scaled,y_train,callbacks=[mc],epochs=20)

Epoch 1/20
804/804 [==============================] - 1s 1ms/step - loss: 0.5729 - accuracy: 0.7220
Epoch 2/20
804/804 [==============================] - ETA: 0s - loss: 0.5604 - accuracy: 0.72 - 1s 999us/step - loss: 0.5601 - accuracy: 0.7273
Epoch 3/20
804/804 [==============================] - 1s 982us/step - loss: 0.5567 - accuracy: 0.7290
Epoch 4/20
804/804 [==============================] - ETA: 0s - loss: 0.5547 - accuracy: 0.7300 ETA: 0s - loss: 0.550 - 1s 998us/step - loss: 0.5552 - accuracy: 0.7294
Epoch 5/20
804/804 [==============================] - 1s 1ms/step - loss: 0.5543 - accuracy: 0.7301
Epoch 6/20
804/804 [==============================] - 1s 1ms/step - loss: 0.5531 - accuracy: 0.7309
Epoch 7/20
804/804 [==============================] - 1s 1ms/step - loss: 0.5520 - accuracy: 0.7324
Epoch 8/20
804/804 [==============================] - 1s 920us/step - loss: 0.5515 - accuracy: 0.7325
Epoch 9/20
804/804 [==============================] - 1s 1ms/step - loss: 0.5512 - a

In [40]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5560 - accuracy: 0.7222 - 335ms/epoch - 1ms/step
Loss: 0.5560042858123779, Accuracy: 0.7222157716751099


In [67]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')

# Model 3

In [41]:
# reduced hidden layers to one
# Initialise the Sequential class
nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=43, input_dim=len(X_train[0]), activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 43)                1806      
                                                                 
 dense_20 (Dense)            (None, 1)                 44        
                                                                 
Total params: 1,850
Trainable params: 1,850
Non-trainable params: 0
_________________________________________________________________


In [42]:
# Compile the model 2
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [43]:
# Train the model
import keras
from keras.callbacks import Callback
mc = keras.callbacks.ModelCheckpoint('opt_weights{epoch:08d}.h5',save_weights_only=True, period=5)
fit_model = nn3.fit(X_train_scaled,y_train,callbacks=[mc],epochs=20)

Epoch 1/20
804/804 [==============================] - 1s 836us/step - loss: 0.5886 - accuracy: 0.7119
Epoch 2/20
804/804 [==============================] - 1s 828us/step - loss: 0.5634 - accuracy: 0.7268
Epoch 3/20
804/804 [==============================] - 1s 959us/step - loss: 0.5594 - accuracy: 0.7278
Epoch 4/20
804/804 [==============================] - 1s 745us/step - loss: 0.5579 - accuracy: 0.7276
Epoch 5/20
804/804 [==============================] - 1s 751us/step - loss: 0.5571 - accuracy: 0.7288
Epoch 6/20
804/804 [==============================] - 1s 1ms/step - loss: 0.5564 - accuracy: 0.7285
Epoch 7/20
804/804 [==============================] - 1s 794us/step - loss: 0.5557 - accuracy: 0.7293
Epoch 8/20
804/804 [==============================] - 1s 846us/step - loss: 0.5550 - accuracy: 0.7305
Epoch 9/20
804/804 [==============================] - 1s 724us/step - loss: 0.5540 - accuracy: 0.7304
Epoch 10/20
804/804 [==============================] - 1s 746us/step - loss: 0.5547 

In [44]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5560 - accuracy: 0.7222 - 178ms/epoch - 666us/step
Loss: 0.5560042858123779, Accuracy: 0.7222157716751099
